<a href="https://colab.research.google.com/github/widyadaridhyp/Diabetes/blob/main/Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/widyadaridhyp/Diabetes/main/diabetes.csv')
df

,num_pregnant,glucose_concentration,blood_pressure,triceps_thickness,two_hour_insulin,bmi,pedigree_function,age,is_diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
from sklearn.model_selection import train_test_split

# Misalkan X adalah dataset fitur (semua kecuali target) dan y adalah target
X = df.drop(columns=['is_diabetes'])  # Sesuaikan dengan nama kolom target yang sesuai
y = df['is_diabetes']

# Memisahkan data menjadi train dan test dengan rasio 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Sekarang Anda dapat menggunakan X_train, X_test, y_train, y_test untuk melatih dan menguji model Anda


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Inisialisasi model Random Forest dengan random_state=42 (opsional)
rf = RandomForestClassifier(random_state=42)

# Latih model dengan data pelatihan
rf.fit(X_train, y_train)

# Lakukan prediksi pada data uji
y_pred = rf.predict(X_test)

# Evaluasi model menggunakan classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.82      0.80      0.81       151
           1       0.64      0.66      0.65        80

    accuracy                           0.75       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.76      0.75      0.75       231



In [6]:
from sklearn.model_selection import GridSearchCV

# Definisikan grid parameter yang akan di-tune
param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion':['gini','entropy']
}



# Inisialisasi GridSearchCV dengan 5-fold cross validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='recall', verbose=1)

# Latih GridSearchCV dengan data pelatihan
grid_search.fit(X_train, y_train)

# Cetak parameter terbaik dan skor terbaik
print("Parameter terbaik:", grid_search.best_params_)
print("Skor Recall terbaik:", grid_search.best_score_)

# Gunakan model terbaik untuk membuat prediksi pada data uji
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Evaluasi model menggunakan classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Parameter terbaik: {'criterion': 'entropy', 'max_features': 'auto'}
Skor Recall terbaik: 0.585348506401138
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       151
           1       0.66      0.64      0.65        80

    accuracy                           0.76       231
   macro avg       0.74      0.73      0.73       231
weighted avg       0.76      0.76      0.76       231



In [7]:
# Setelah GridSearchCV selesai
best_rf = grid_search.best_estimator_

# Latih kembali model terbaik dengan data pelatihan
best_rf.fit(X_train, y_train)

# Dapatkan nilai penting fitur
feature_importances = best_rf.feature_importances_

# Buat DataFrame untuk menampilkan feature importances
importances_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Urutkan berdasarkan Importance secara menurun
importances_df = importances_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Ambil top 3 fitur yang paling penting
top_3_features = importances_df.head(3)

print("Top 3 Feature Importance:")
print(top_3_features)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Top 3 Feature Importance:
                 Feature  Importance
0  glucose_concentration    0.260577
1                    bmi    0.165501
2                    age    0.147795


In [16]:
importances_df

,Feature,Importance
0,glucose_concentration,0.260577
1,bmi,0.165501
2,age,0.147795
3,pedigree_function,0.107276
4,blood_pressure,0.096157
5,num_pregnant,0.082889
6,two_hour_insulin,0.070521
7,triceps_thickness,0.069284
